## Memory
In some applications like chatbots, it is important to remember previous interactions to keep the whole context of a conversation.
Memory does provide you an easy way to handle this

In [2]:
import os
import openai 
from dotenv import load_dotenv, find_dotenv
from langchain import PromptTemplate, OpenAI, LLMChain

load_dotenv(find_dotenv()) 
openai.api_key = os.environ['OPENAI_API_KEY']

In [4]:
from langchain.memory import ChatMessageHistory
history = ChatMessageHistory()

history.add_user_message("hi!")
history.add_ai_message("hello my friend!")
history.messages

[HumanMessage(content='hi!', additional_kwargs={}, example=False),
 AIMessage(content='hello my friend!', additional_kwargs={}, example=False)]

In [5]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("hello my friend!")
memory.load_memory_variables({})

{'history': 'Human: hi!\nAI: hello my friend!'}

In [6]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)
conversation.predict(input="Hi")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi
AI:

> Finished chain.


' Hi there! How can I help you?'

In [7]:
conversation.predict(input="I need to know the capital of france")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi
AI:  Hi there! How can I help you?
Human: I need to know the capital of france
AI:

> Finished chain.


' The capital of France is Paris. It is located in the northern part of the country and is the most populous city in France. It is also the cultural and economic center of the country.'

## ConversationSummaryMemory
When inputs get long, we might not want to send the whole conversation, but rather a summary.

In [9]:
!pip install tiktoken

  Using cached tiktoken-0.4.0-cp311-cp311-win_amd64.whl (635 kB)


You should consider upgrading via the 'C:\Users\User\.virtualenvs\autogpt\Scripts\python.exe -m pip install --upgrade pip' command.


In [10]:
from langchain.memory import ConversationSummaryBufferMemory

review = "I ordered Pizza Salami for 9.99$ and it was awesome! \
The pizza was delivered on time and was still hot when I received it. \
The crust was thin and crispy, and the toppings were fresh and flavorful. \
The Salami was well-cooked and complemented the cheese perfectly. \
The price was reasonable and I believe I got my money's worth. \
Overall, I am very satisfied with my order and I would recommend this pizza place to others."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello, how can I help you today?"}, {"output": "Could you analyze a review for me?"})
memory.save_context({"input": "Sure, I'd be happy to. Could you provide the review?"}, {"output": f"{review}"})

In [11]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [12]:
conversation.predict(input="Thank you very much!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human asked how they could help the AI, and the AI requested an analysis of a review. The human agreed to provide the analysis.
AI: I ordered Pizza Salami for 9.99$ and it was awesome! The pizza was delivered on time and was still hot when I received it. The crust was thin and crispy, and the toppings were fresh and flavorful. The Salami was well-cooked and complemented the cheese perfectly. The price was reasonable and I believe I got my money's worth. Overall, I am very satisfied with my order and I would recommend this pizza place to others.
Human: Thank you very much!
AI:

> Finished chain.


" You're welcome! It was my pleasure to share my experience with you. Is there anything else I can help you with?"

In [13]:
memory.load_memory_variables({})

{'history': "System: \n\nThe human asked how they could help the AI, and the AI requested an analysis of a review. The human agreed to provide the analysis, which was a review of a Pizza Salami order for 9.99$ that was delivered on time and was still hot when received. The crust was thin and crispy, the toppings were fresh and flavorful, and the Salami was well-cooked and complemented the cheese perfectly. The price was reasonable and the customer was very satisfied with their order, recommending the pizza place to others.\nHuman: Thank you very much!\nAI:  You're welcome! It was my pleasure to share my experience with you. Is there anything else I can help you with?"}